In [1]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import os

In [2]:
df = pd.read_csv("../data/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
facts_template = """
1. Number of times pregnant is {}.
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test is {}.
3. Diastolic blood pressure (mm Hg) is {}.
4. Triceps skin fold thickness (mm) is {}.
5. 2-Hour serum insulin (mu U/ml) is {}.
6. Body mass index (weight in kg/(height in m)^2) is {}.
7. Diabetes pedigree function is {}.
8. Age (years) is {}.
"""

In [25]:
df.iloc[1]

Pregnancies                  1.000
Glucose                     85.000
BloodPressure               66.000
SkinThickness               29.000
Insulin                      0.000
BMI                         26.600
DiabetesPedigreeFunction     0.351
Age                         31.000
Outcome                      0.000
Name: 1, dtype: float64

In [35]:
example_patient = facts_template.format(df.iloc[1]["Pregnancies"],
                    df.iloc[1]["Glucose"], 
                    df.iloc[1]["BloodPressure"],
                    df.iloc[1]["SkinThickness"],
                    df.iloc[1]["Insulin"],
                    df.iloc[1]["BMI"],
                    df.iloc[1]["DiabetesPedigreeFunction"],
                    df.iloc[1]["Age"])

In [36]:
example_patient

'\n1. Number of times pregnant is 1.0.\n2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test is 85.0.\n3. Diastolic blood pressure (mm Hg) is 66.0.\n4. Triceps skin fold thickness (mm) is 29.0.\n5. 2-Hour serum insulin (mu U/ml) is 0.0.\n6. Body mass index (weight in kg/(height in m)^2) is 26.6.\n7. Diabetes pedigree function is 0.351.\n8. Age (years) is 31.0.\n'

# Generation

In [37]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

In [38]:
def chat_completetion(messages, client, model):
    response = client.chat.completions.create(
    model=model,
    stream=False,
    messages=messages
    )

    return response.choices[0].message.content

In [39]:
# gpt call
client = OpenAI(api_key=API_KEY)

SYSTEM ="""
        - You are an assistant for a medical company 
        - Your objective is to analyse and diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements.
        - You should format you output in a form of a lengthy report analysing and exmplaining the significanse of each measurments and how the measurments might affect you final jugment.
        """

MESSAGES = [
    {"role": "system", "content": SYSTEM},
    {"role": "user", "content": "Give me a report of the patient status given theses mesurments: {}".format(example_patient)},
]

MODEL= "gpt-4o"
response = chat_completetion(MESSAGES,client, MODEL)

In [40]:
# Writing to file
with open("../examples/example.txt", "w") as file1:
    # Writing data to a file
    file1.write(response)